<a href="https://colab.research.google.com/github/kyungminkim-dev/boostcamp-ai-tech/blob/main/2_word2vec_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 52.9MB/s 
     |████████████████████████████████| 92kB 16.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [4]:
tokenizer = Okt()

In [5]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [6]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


In [7]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 15557.51it/s]


In [8]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [9]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 238764.93it/s]


In [10]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [11]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [12]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [13]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 4401.16it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [15]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [16]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [19]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [20]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [40]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/16 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.80348014831543
Train loss: 4.353360176086426
Train loss: 5.988618850708008
Train loss: 4.567574501037598
Train loss: 5.384766578674316
Train loss: 4.348655700683594
Train loss: 5.378296375274658
Train loss: 5.190069198608398
Train loss: 4.461023330688477
Train loss: 4.014639854431152
Train loss: 3.912533760070801
Train loss: 3.9667863845825195
Train loss: 4.599996566772461
Train loss: 5.2390947341918945
Train loss: 4.460725784301758
Train loss: 5.714531898498535
##################################################
Epoch: 2
Train loss: 4.6101861000061035
Train loss: 4.210658073425293
Train loss: 5.827690124511719
Train loss: 4.439815521240234
Train loss: 5.252415657043457
Train loss: 4.071657657623291
Train loss: 5.203367233276367
Train loss: 5.058775901794434
Train loss: 4.344847679138184
Train loss: 3.8284854888916016
Train loss: 3.748732805252075
Train loss: 3.5951831340789795
Train loss: 4.458020210266113
Train 

100%|██████████| 16/16 [00:00<00:00, 614.66it/s]

Train loss: 2.937896490097046
Train loss: 4.180788516998291
Train loss: 4.8955488204956055
Train loss: 4.043193817138672
Train loss: 5.293586254119873
##################################################
Epoch: 5
Train loss: 4.067689895629883
Train loss: 3.80252742767334
Train loss: 5.360133647918701
Train loss: 4.066681861877441
Train loss: 4.864297389984131
Train loss: 3.3425750732421875
Train loss: 4.69443941116333
Train loss: 4.675711154937744
Train loss: 4.033030033111572
Train loss: 3.3198599815368652
Train loss: 3.3251118659973145
Train loss: 2.6598780155181885
Train loss: 4.045436382293701
Train loss: 4.785314559936523
Train loss: 3.907994270324707
Train loss: 5.158686637878418
Finished.


다음으로 Skip-gram 모델 학습입니다.

In [43]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 3.7816238403320312
Train loss: 4.267120361328125
Train loss: 4.215713024139404
Train loss: 4.752714157104492
Train loss: 4.187987327575684
Train loss: 4.019939422607422
Train loss: 4.580994129180908
Train loss: 4.206258773803711
Train loss: 4.453839302062988
Train loss: 4.290581703186035
Train loss: 3.7114858627319336
Train loss: 4.110230922698975
Train loss: 4.023531913757324
Train loss: 4.344503879547119
Train loss: 3.731412410736084
Train loss: 4.101868629455566
Train loss: 3.892960786819458
Train loss: 4.536526203155518
Train loss: 4.085565567016602
Train loss: 3.7526304721832275
Train loss: 4.616669654846191
Train loss: 3.783750057220459
Train loss: 4.68772554397583
Train loss: 4.172626972198486
Train loss: 3.9627764225006104
Train loss: 4.725791931152344
Train loss: 4.18694543838501
Train loss: 4.426153182983398
Train loss: 4.216753959655762
Train loss: 3.9124090671539307
Train loss: 4.282020092010498
Train l

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.042933940887451
Train loss: 3.689474582672119
Train loss: 4.388166427612305
Train loss: 3.6338419914245605
Train loss: 4.574385643005371
Train loss: 4.1124749183654785
Train loss: 3.8962178230285645
Train loss: 4.586732864379883
Train loss: 4.079136371612549
Train loss: 4.367153167724609
Train loss: 4.143271446228027
Train loss: 3.857530355453491
Train loss: 4.226344585418701
Train loss: 4.509611129760742
Train loss: 4.194073677062988
Train loss: 4.37539005279541
Train loss: 4.227226257324219
Train loss: 4.140751838684082
Train loss: 4.224636077880859
Train loss: 4.0549116134643555
Train loss: 4.032726764678955
Train loss: 4.212347030639648
Train loss: 3.8922855854034424
Train loss: 4.290648460388184
Train loss: 3.9344756603240967
Train loss: 4.136038780212402
Train loss: 3.961487293243408
Train loss: 4.152508735656738
Train loss: 4.1172404289245605
Train loss: 4.364009380340576
Train loss: 3.9875025749206543
Train loss: 3.670264720916748
Train loss: 3.866649866104126
Tra

100%|██████████| 64/64 [00:00<00:00, 744.54it/s]

Train loss: 3.9466466903686523
Train loss: 4.160982131958008
Train loss: 3.8556268215179443
Train loss: 4.249560832977295
Train loss: 3.8356518745422363
Train loss: 4.049046516418457
Train loss: 3.7576000690460205
Train loss: 3.935391426086426
Train loss: 3.974210500717163
Train loss: 4.255073070526123
Train loss: 3.921862840652466
Train loss: 3.619279623031616
Train loss: 3.795088768005371
Train loss: 4.381435394287109
Train loss: 4.199204444885254
Train loss: 4.024593830108643
Train loss: 4.02753210067749
Train loss: 3.8334665298461914
Train loss: 4.08950138092041
Train loss: 4.241134166717529
Train loss: 4.369747638702393
Train loss: 4.594306468963623
Train loss: 3.5445523262023926
Train loss: 4.116828918457031
Train loss: 3.7445807456970215
Train loss: 4.30820369720459
Finished.


### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [44]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-3.8133e-01,  1.5533e-01,  1.4973e+00,  2.2798e+00,  7.4904e-02,
         1.4004e+00,  2.5119e-01, -1.7012e+00, -4.3473e-01, -3.5296e-01,
         4.9321e-01, -9.9620e-01, -6.9636e-01,  9.5526e-01, -2.5815e-04,
        -8.0536e-01,  9.9559e-01, -2.0495e+00, -1.3111e+00, -1.5635e-01,
         7.6210e-01, -7.4923e-01, -4.8993e-01, -6.7755e-01,  7.3313e-01,
        -1.8821e-01,  7.8118e-01, -1.0952e+00, -4.2046e-01, -1.7938e+00,
         1.1852e-01, -1.9163e-01, -2.6665e-01, -2.0962e-01,  9.5891e-01,
         7.8954e-02, -5.7245e-01,  9.6510e-02, -1.4133e+00, -4.5091e-01,
         5.6576e-01,  1.1070e+00,  2.0158e+00,  8.6939e-03, -4.9678e-01,
         5.7098e-01, -8.4483e-02,  2.5549e-02,  1.2124e+00,  2.4845e-01,
        -4.6574e-01,  4.3416e-02,  7.1051e-01,  8.2970e-01,  7.2512e-01,
        -1.6431e+00,  1.1731e+00, -6.4687e-01,  1.6348e+00, -8.4508e-01,
         2.6567e-01,  3.5450e-01,  3.8265e-01, -5.5076e-01,  8.2978e-01,
        -6.9080e-01,  9.1097e-01, -4.7613e

In [45]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(max(emb.squeeze(0)))

Word: 음식
tensor(2.4956, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 맛
tensor(2.5353, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 서비스
tensor(2.5055, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 위생
tensor(3.3142, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 가격
tensor(3.4253, device='cuda:0', grad_fn=<UnbindBackward>)


In [46]:
!apt-get install -qq texlive texlive-xetex texlive-latex-extra pandoc
!pip install -qq pypandoc

from google.colab import drive
drive.mount('/content/drive')

!jupyter nbconvert --to PDF '/content/drive/My Drive/Colab Notebooks/1_naive_bayes.ipynb의 사본'

Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2_all.deb ...
Unpacking fonts-lato (2.0-2) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.8-2_all.deb ...
Unpacking poppler-data (0.4.8-2) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.09_all.deb ...
Unpacking tex-common (6.09) ...
Selecting previously unselected package fonts-lmodern.
Preparing to unpack .../04-fonts-lmodern_2.004.5-3_all.deb ...
Unpacking fonts-lmodern (2.004.5-3) ...
Selecting previously unselected package fonts-noto-mono.
Preparing to unpack .